In [402]:
import jsonlines
def jsonl_to_data(filename):
    data = {}
    with jsonlines.open(filename) as reader:
        for line in reader:
            try:
                data[line['id']] = line
            except Exception as ex:
                print(ex, line)
    return data

In [403]:
from unidecode import unidecode
import string
def to_english(s):
    s = list(str(s))
    for i in range (len(s)):
        s[i] = unidecode(s[i])
    s1 = ""
    for i in s:
        s1+=i
    s1 = s1.lower()
    s1 = s1.translate(str.maketrans('', '', string.punctuation))
    return " ".join(s1.split())

In [404]:
import fasttext
model_school = fasttext.load_model('model_ex.bin')
model_major = fasttext.load_model('model_ex_major.bin')

In [405]:
data = jsonl_to_data('50k_candidates_for_Tri.jsonl')

In [406]:
import pandas as pd
df = pd.DataFrame(data)

In [407]:
df = df.transpose()

In [408]:
df = df[df['education'].map(lambda d: len(d)) > 0]
df = df[df['work_history'].map(lambda d: len(d)) > 0]

In [409]:
p_data = dict()
p_data['degree'] = []
p_data['school'] = []
p_data['major'] = []
p_data['education_period'] = []
p_data['work_title'] = []
p_data['company'] = []
p_data['work_period'] = []
for ind in df.index:
    education = df.loc[ind]['education']
    for i in education:
        for j in df.loc[ind]['work_history']:
            p_data['work_title'].append(j['title'])
            p_data['company'].append(j['company'])
            p_data['work_period'].append(j['period'])
            p_data['degree'].append(i['degree'])
            p_data['school'].append(i['school'])
            p_data['major'].append(i['major'])
            p_data['education_period'].append(i['period'])
p_data = pd.DataFrame(p_data)
print(p_data)                   

                 degree                                  school  \
0              Cao Đẳng                 Cao Đẳng Du Lịch Hà Nội   
1              Cao Đẳng                 Cao Đẳng Du Lịch Hà Nội   
2              Cao Đẳng                 Cao Đẳng Du Lịch Hà Nội   
3               Đại Học  Trường Đại học Kinh tế tp. Hồ Chí Minh   
4               Đại Học  Trường Đại học Kinh tế tp. Hồ Chí Minh   
...                 ...                                     ...   
54911           Đại Học                        Đại học Thủy Lợi   
54912           Đại Học              Đại học Công Nghiệp TP.HCM   
54913           Đại Học              Đại học Công Nghiệp TP.HCM   
54914  Trung cấp - Nghề                                    THPT   
54915  Trung cấp - Nghề                                    THPT   

                       major    education_period  \
0        Quản trị kinh doanh      [-01, 2015-01]   
1        Quản trị kinh doanh      [-01, 2015-01]   
2        Quản trị kinh doanh      [-01,

In [410]:
p_data = p_data.drop(['degree'],axis = 1)

In [411]:
p_data = p_data.dropna()

In [412]:
school_predicted = []
school_confidence = []
major_predicted = []
major_confidence = []
for ind in p_data.index:
    school_predicted.append(model_school.predict(to_english(p_data.loc[ind]['school']))[0][0][9:].replace("-"," ").replace("   "," - "))
    school_confidence.append(model_school.predict(to_english(p_data.loc[ind]['school']))[1][0])
    major_predicted.append(model_major.predict(to_english(p_data.loc[ind]['major']))[0][0][9:].replace("-"," ").replace("   "," - "))
    major_confidence.append(model_major.predict(to_english(p_data.loc[ind]['major']))[1][0])


In [413]:
p_data["school_predicted"] = school_predicted
p_data['school_confidence'] = school_confidence
p_data['major_predicted'] = major_predicted
p_data['major_confidence'] = major_confidence

In [414]:
f_data = p_data[(p_data.school_confidence>0.9) &  (p_data.major_confidence>0.9)]

In [415]:
f_data = f_data.reset_index(drop = True)

In [416]:
start_date_school = []
end_date_school = []
for ind in f_data.index:
    start_date_school.append(f_data.loc[ind]['education_period'][0])
    end_date_school.append(f_data.loc[ind]['education_period'][1])
f_data['start_date_school'] = start_date_school
f_data['end_date_school'] = end_date_school

In [417]:
start_date_work = []
end_date_work = []
for ind in f_data.index:
    start_date_work.append(f_data.loc[ind]['work_period'][0])
    end_date_work.append(f_data.loc[ind]['work_period'][1])
f_data['start_date_work'] = start_date_work
f_data['end_date_work'] = end_date_work

In [418]:
import re
import datetime
import dateutil.parser as parser 
end_year_school = []
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
for ind in f_data.index:
    if f_data.loc[ind]['end_date_school'] == 'None' or f_data.loc[ind]['end_date_school']=='-1':
        check = False
        track = f_data.loc[ind]['start_date_school'].replace('/',' ').replace('-',' ').split()
        for i in range(len(track)):
            if len(track[len(track)-1-i])==4 and is_number(track[len(track)-1-i]):
                end_year_school.append(int(track[len(track)-1-i]))
                check = True
                break
        if not check:
            end_year_school.append(-1)
    else:
        check = False
        track = f_data.loc[ind]['end_date_school'].replace('/',' ').replace('-',' ').split()
        for i in range(len(track)):
            if len(track[len(track)-1-i])==4 and is_number(track[len(track)-1-i]):
                end_year_school.append(int(track[len(track)-1-i]))
                check = True
                break
        if not check:
            end_year_school.append(-1)

In [419]:
f_data['end_year_school'] = end_year_school 

In [420]:
f_data = f_data[f_data.end_year_school>0]

In [421]:
start_year_work = []
end_year_work = []
for ind in f_data.index:
    if f_data.loc[ind]['start_date_work'] == 'None' or f_data.loc[ind]['start_date_work'] == '-01':
        check_s = False
        check_e = False
        track = f_data.loc[ind]['end_date_work'].replace('/',' ').replace('-',' ').split()
        for i in range(len(track)):
            if len(track[i])==4 and is_number(track[i]):
                start_year_work.append(int(track[i]))
                check_s = True
                break
        for i in range(len(track)):
            if len(track[len(track)-1-i])==4 and is_number(track[len(track)-1-i]):
                end_year_work.append(int(track[len(track)-1-i]))
                check_e = True
                break
        if check_s == False:
            start_year_work.append(-1)
        if check_e == False:
            end_year_work.append(-1)
    elif f_data.loc[ind]['end_date_work'] == 'None' or f_data.loc[ind]['end_date_work'] == '-01':
        check_s = False
        check_e = False
        track = f_data.loc[ind]['start_date_work'].replace('/',' ').replace('-',' ').split()
        for i in range(len(track)):
            if len(track[i])==4 and is_number(track[i]):
                start_year_work.append(int(track[i]))
                check_s = True
                break
        for i in range(len(track)):
            if len(track[len(track)-1-i])==4 and is_number(track[len(track)-1-i]):
                end_year_work.append(int(track[len(track)-1-i]))
                check_e = True
                break
        if not check_s:
            start_year_work.append(-1)
        if not check_e:
            end_year_work.append(-1)
    else:
        check_s = False
        check_e = False
        track = f_data.loc[ind]['start_date_work'].replace('/',' ').replace('-',' ').split()
        for i in range(len(track)):
            if len(track[i])==4 and is_number(track[i]):
                start_year_work.append(int(track[i]))
                check_s = True
                break
        track = f_data.loc[ind]['end_date_work'].replace('/',' ').replace('-',' ').split()
        for i in range(len(track)):
            if len(track[len(track)-1-i])==4 and is_number(track[len(track)-1-i]):
                end_year_work.append(int(track[len(track)-1-i]))
                check_e = True
                break
        if not check_s:
            start_year_work.append(-1)
        if not check_e:
            end_year_work.append(-1)
f_data['start_year_work'] = start_year_work
f_data['end_year_work'] = end_year_work

In [422]:
f_data = f_data[(f_data.start_year_work>1900) & (f_data.end_year_work>1900)]

In [ ]:
year_category = []
for ind in f_data.index:
    grad = f_data.loc[ind]['end_year_school']
    start = f_data.loc[ind]['start_year_work']
    end = f_data.loc[ind]['end_year_work']
    res = []
    if start <= grad +1:
        res.append(0)
    if start <=grad + 3 and end>= grad + 3:
        res.append(3)
    if start <= grad + 5 and end >= grad + 5:
        res.append(5)
    year_category.append(res)
f_data['year_category'] = year_category

In [427]:
f_data.to_csv('50K_school_major_year.csv')

In [426]:
print(f_data)

                                       school                  major  \
0                     Cao Đẳng Du Lịch Hà Nội    Quản trị kinh doanh   
1                     Cao Đẳng Du Lịch Hà Nội    Quản trị kinh doanh   
2                     Cao Đẳng Du Lịch Hà Nội    Quản trị kinh doanh   
3      Trường Đại học Kinh tế tp. Hồ Chí Minh    Quản trị kinh doanh   
4      Trường Đại học Kinh tế tp. Hồ Chí Minh    Quản trị kinh doanh   
...                                       ...                    ...   
22755                        Đại học Thủy Lợi       Kế toán, Kế toán   
22756                        Đại học Thủy Lợi       Kế toán, Kế toán   
22757                        Đại học Thủy Lợi       Kế toán, Kế toán   
22758              Đại học Công Nghiệp TP.HCM  Tài Chính – Ngân Hàng   
22759              Đại học Công Nghiệp TP.HCM  Tài Chính – Ngân Hàng   

      education_period                                         work_title  \
0       [-01, 2015-01]                                    

2024-01
